## Meteo Bakery - LightGBM feature importance
In this notebook, we will test LightGBM algorithm and assess feature importance.

### import libraries

In [ ]:
# import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit

from lightgbm import LGBMRegressor

### load data

In [ ]:
df = pd.read_csv('../data/data_combined.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.head()

### transform periodic month feature using sine and cosine functions

In [ ]:
df['month_sin'] = df.month.apply(lambda x: np.sin(np.array(x) * np.pi /6))
df['month_cos'] = df.month.apply(lambda x: np.cos(np.array(x) * np.pi /6))

### select only years up to 2020

In [ ]:
df = df[df.year<2020]

### generate lag features

In [ ]:
# initialize empty dataframe
df_lag = pd.DataFrame({})
for i, group in enumerate(product(df['branch'].unique(), df['product'].unique())):
        # subselect time series and generate lag features
        ts = df[(df['branch']==group[0]) & (df['product']==group[1])].copy()
        # map feature to dictionary
        target_map = ts['turnover'].to_dict()
        ts['lag_07'] = (ts.index - pd.Timedelta('7 days')).map(target_map)
        ts['lag_365'] = (ts.index - pd.Timedelta('365 days')).map(target_map)
        # concatenate to dataframe
        df_lag = pd.concat([df_lag, ts], axis=0)

In [ ]:
df_lag.sort_index(inplace=True)

In [ ]:
df_lag.head()

### check missings in sales data

In [ ]:
df_lag.groupby(['branch', 'product'])['turnover', 'month'].count()

In [ ]:
df_lag[(df_lag['turnover'].isnull()) & (df_lag['branch']=='Metro')]

4 missing days for Metro station. Additionally, there no sales for Mischbrote on 16-10-2018 and 16-10-2019.

In [ ]:
df_lag[(df_lag['turnover'].isnull()) & (df_lag['branch']=='Train_Station')]

Train Station has exactly the same missings as Metro branch.

In [ ]:
df_lag[(df_lag['turnover'].isnull()) & (df_lag['branch']=='Center') & (df_lag['product']=='Mischbrote')].head(25)

69 missing days for Center branch. There frequently fall on a public holiday, thus indicating that this branch probably had closed on these days.

### replace NaNs
replace NaNs in turnover column with turnover of corresponding day of preceding weak, otherwise, use combined forward fill

In [ ]:
df_repl = df_lag.copy()
# replace NaN at Center branch by 1 is occurring on public holiday
df_repl.loc[(df_repl['branch']=='Center') & (df_repl['public_holiday']==True), 'turnover'] = df_repl.loc[(df_repl['branch']=='Center') & (df_repl['public_holiday']==True), 'turnover'].fillna(1)

# fill NaN with sales from previous day of week
df_repl['turnover'] = df_repl['turnover'].fillna(df_repl['lag_07'])

# fill remaining NaN using forward fill
for i, group in enumerate(product(df_repl['branch'].unique(), df_repl['product'].unique())):
        df_repl[(df['branch']==group[0]) & (df_repl['product']==group[1])].ffill(inplace=True, axis='rows')

df_repl.head()

In [ ]:
df_repl.loc[(df_repl['branch']=='Metro') & (df_repl['product']=='Mischbrote'), ['branch', 'product', 'turnover', 'lag_07']].head(20)

In [ ]:
df_repl.groupby(['branch', 'product'])[['turnover', 'lag_07', 'month']].count()

In [ ]:
df_repl[(df_repl['public_holiday']==True) & (df_repl['branch']=='Center')]

### generate train and test df

In [ ]:
df_train = df_repl[df_repl.year<2018]
df_test = df_repl[df_repl.year>=2018]

### Time Series Split Cross Validation
Number of splits is set to 52 and test sizeto 7 days, thus representing a whole year. For now, we won't introduce any gab between training and validation set. However, assuming that bakery stores usually make their demand planning for the upcoming week on certain days of the week, e.g. Thursdays, we may later additionally introduce a gap of 3 days.

In [ ]:
# extract an example time series for illustration purposes and perform TimeSeriesSplit
ts = df_train[(df_train['branch']=='Metro') & (df_train['product']=='Mischbrote')]['turnover']
tss = TimeSeriesSplit(n_splits=52, test_size=7, gap=3)

fold=0
# plot repeated train-validation folds to get an idea of TimeSeriesSplit functionality
for train_i, val_i in tss.split(ts):
    ts_train = ts.iloc[train_i]
    ts_val = ts.iloc[val_i]

    plt.figure(figsize=(10, 1))
    ts_train[-500:].plot(c='blue', label='training')
    ts_val.plot(c='red', label='validation')

plt.show()

### Naive Seasonal baseline
We will first define a utility function to perform cross-validation on naive seasonal baseline.

#### use sales 14 days ago if prediction contains missing from closed Center branch on public holidays
It is possible that a prediction for Naive Seasonal baseline includes values == 1 as prediction, which relfected NaN from public holidays in the preceding 7 day-interval used for prediction in case of Center branch.

In such as case, replace any NaN (currently encoded as 1) by the turnover day from exactly 7 days ago. Thus, in case a holiday is contained in the preceding week before prediction, any such holiday is replaced by sales data from the day of the preceding week. Thus, not the sales 7 days before before are used as a prediction when falling on a holiday, but instead the sales 14 days ago.

#### also correct validation set if necessary
Find missings in validation set due to holiday-related closing (represented as 1), extract index positions and drop elements at these index positions from both validation and training set.

In [ ]:
# utility function for naive seasonal baseline corrected for holiday effects
def naive_seasonal_cv_holiday(df_train, grouping_features, target, gap=0):
    # initialize dataframe for evaluation scores
    cval = pd.DataFrame({'group': [], 'MAPE': []})

    # iterate over all individual series and perform cross-validation
    from itertools import product
    for i, group in enumerate(product(df_train[grouping_features[0]].unique(), df_train[grouping_features[1]].unique())):

        # subselect time series
        ts = df_train[(df_train[grouping_features[0]]==group[0]) & (df_train[grouping_features[1]]==group[1])].copy()

        # perform cross validation
        tss = TimeSeriesSplit(n_splits=52, test_size=7, gap=gap)

        scores = []
        for train_i, val_i in tss.split(ts):

            target = 'turnover'
            y_train = ts.iloc[train_i][target]
            y_val = ts.iloc[val_i][target]
            
            # correct for holiday effects in predicted values based on training set if necessary
            # if holiday is in prediced y-values, replace by sales 14 days ago
            if 1 in y_train[-7:].unique():
                idx_train = [i for i in range(len(y_train[-7:].tolist())) if y_train[-7:].tolist()[i]==1]
                idx_train = [i-7 for i in idx_train]
                idx_train_lag = [i-7 for i in idx_train]
                y_train_repl = y_train.copy()
                y_train_repl.iloc[idx_train] = [x for x in y_train.iloc[idx_train_lag]]
                y_pred = y_train_repl[-7:]
            else:
                y_pred = y_train[-7:]

            # correct for holiday effects in validation set if necessary
            # if holiday is in validation set, drop elements at corresponding index position in both y_val and y_pred
            if 1 in y_val.unique():
                idx_val = [i for i in range(len(y_val.tolist())) if y_val.tolist()[i]==1]
                y_val = y_val.drop(y_val.index[idx_val])
                y_pred = y_pred.drop(y_pred.index[idx_val])

            mape = mean_absolute_percentage_error(y_val, y_pred)
            scores.append(mape)
        
        # append scores
        cval.loc[i, 'group'] = f'{group[0]} | {group[1]}'
        cval.loc[i, 'MAPE'] = np.mean(scores)
    # calculate mean scores
    cval.loc[i+1, 'group'] = 'mean'
    cval.loc[i+1, 'MAPE'] = cval['MAPE'].mean()

    return cval

In [ ]:
scores_naive = naive_seasonal_cv_holiday(df_train, grouping_features=['branch', 'product'], target='turnover')

In [ ]:
scores_naive

### LightGBM
We will iterate over every time series and evaluate LightGBM performance with only temporal and additional weather features using TimeSeriesSplit Cross-Validation.

First, we will define a utility function performing Cross-Validation.

In [ ]:
# utility function for LightGBM
def LGBM_cv(df_train, grouping_vars, target, features, gap=0):
    # initialize dataframe for evaluation scores
    cval = pd.DataFrame({'group': [], 'MAPE': []})

    # iterate over all individual series and perform cross-validation
    from itertools import product
    for i, group in enumerate(product(df_train[grouping_vars[0]].unique(), df_train[grouping_vars[1]].unique())):

        # subselect time series
        ts = df_train[(df_train[grouping_vars[0]]==group[0]) & (df_train[grouping_vars[1]]==group[1])].copy()

        # perform cross validation
        tss = TimeSeriesSplit(n_splits=52, test_size=7, gap=gap)

        scores = []
        for train_i, val_i in tss.split(ts):

            train = ts.iloc[train_i]
            val = ts.iloc[val_i]
                    
            X_train = train[features]
            X_val= val[features]
            y_train = train[target]
            y_val = val[target]

            lgbm = LGBMRegressor(objective='regression', random_state=42)
            lgbm.fit(X_train, y_train)

            y_pred = pd.Series(lgbm.predict(X_val))

            # correct for holiday effects in validation set if necessary
            # if holiday is in validation set, drop elements at corresponding index position in both y_val and y_pred
            if 1 in y_val.unique():
                idx_val = [i for i in range(len(y_val.tolist())) if y_val.tolist()[i]==1]
                y_val = y_val.drop(y_val.index[idx_val])
                y_pred = y_pred.drop(y_pred.index[idx_val])

            mape = mean_absolute_percentage_error(y_val, y_pred)
            scores.append(mape)
        
        # append scores
        cval.loc[i, 'group'] = f'{group[0]} | {group[1]}'
        cval.loc[i, 'MAPE'] = np.mean(scores)
    # calculate mean scores
    cval.loc[i+1, 'group'] = 'mean'
    cval.loc[i+1, 'MAPE'] = [cval['MAPE'].mean()]
    
    return cval

### temporal features

In [ ]:
time_features = ['lag_07', 'lag_365', 'month_sin', 'month_cos', 'day_of_week', 'school_holiday', 'public_holiday']

In [ ]:
scores_lgbm_time = LGBM_cv(df_train, grouping_vars=['branch', 'product'], target='turnover', features=time_features)

In [ ]:
scores_lgbm_time

### basic weather statistics (temperature, humidity, rain, snow)
Here, will will add daily weather aggregate features as predictors to assess any add-on effect in addition to the temporal features. We will use mean temperature, humidity, rain, and snow as weather features since they appear most promising based on previous EDA results.

In [ ]:
mean_weather_features = ['lag_07', 'lag_365', 'month_sin', 'month_cos', 'day_of_week', 'school_holiday', 'public_holiday',
                                    'temp_mean', 'humidity_mean', 'rain_1h_mean', 'snow_1h_mean']

In [ ]:
scores_lgbm_weather = LGBM_cv(df_train, grouping_vars=['branch', 'product'], target='turnover', features=mean_weather_features)

In [ ]:
scores_lgbm_weather

### basic weather statistics and climatological days
In addition to aggregated features, we calculated features for climatological days according to [DWD](https://www.dwd.de/DE/service/lexikon/Functions/glossar.html;jsessionid=EB2D3A27D634826A0176255436956DA7.live21064?lv2=101334&lv3=101452) based on our weather statistics. We first performed some basic EDA to test which climatological days could serve as potential predictors in the LGBM model.

#### visualize relative occurrence of climatological days depending on month

In [ ]:
for day in ['day_icy', 'day_frosty', 'day_thunder', 'day_hot', 'day_clear','day_hazy', 'day_rainy', 'day_summer', 'day_murky']:
    plt.figure(figsize=(7,1))
    sns.barplot(data=df, y=day, x='month', color='white', edgecolor='blue')
    plt.yticks(ticks=np.arange(0, 0.81, 0.2));

#### visualize differences in sales for different product categories depending on climatological days

In [ ]:
for day in ['day_icy', 'day_frosty', 'day_thunder', 'day_hot', 'day_clear','day_hazy', 'day_rainy', 'day_summer', 'day_murky']:
    plt.figure(figsize=(5, 2))
    sns.barplot(data=df, x='product', y='turnover', edgecolor='blue', hue=day)
    plt.xticks(rotation=45, ha='right')
    plt.legend(title=day, bbox_to_anchor=(1.05, 1.0), loc='upper left')

Day frosty and Day icy measure similar weather conditions and have similar effects. The same holds true for day hot and day summer. However, day frosty and day summer have higher occurrence, so we will use these ones as predictors, as opposed to the other ones. 

Day rainy has almost no occurrence and almost no effect and is therefore not used as predictor. Day murky is also not used since it doesn´t have any clear effect and represents the counterpart to day clear. 

Day clear and day hazy don´t seem to have clear effects either, but are included as predictors at this stage.

In [ ]:
mean_weather_climat_features = ['lag_07', 'lag_365', 'month_sin', 'month_cos', 'day_of_week', 'school_holiday', 'public_holiday',
                                    'temp_mean', 'humidity_mean', 'rain_1h_mean', 'snow_1h_mean', 'day_frosty', 'day_thunder', 'day_clear','day_hazy', 'day_summer']

In [ ]:
scores_lgbm_climat = LGBM_cv(df_train, grouping_vars=['branch', 'product'], target='turnover', features=mean_weather_climat_features)

In [ ]:
scores_lgbm_climat

### basic weather statistics, climatological days and elative frequency measures of weather states
Finally, we will also test whether the inclusion of relative frequency of defined weather states affects forecasting performance. To this end, we will first perform some feature pre-selection based on basic EDA.

#### visualize differences in sales for different product categories depending relative frequencies of weather states

In [ ]:
for x in ['clear_total', 'cloudy_total', 'foggy_total', 'rainy_total', 'snowy_total', 'thunderstorm_total', 'tornado_total']:
    fig, ax = plt.subplots(figsize=(9, 2))
    from matplotlib.ticker import FormatStrFormatter
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    sns.barplot(data=df, x=x, y='turnover', edgecolor='blue', hue='product', ax=ax)
    xlabels = ['{:,.2f}'.format(x) for x in ax.get_xticks()]
    ax.set_xticklabels(xlabels, rotation=45)
    plt.legend(fontsize=7, bbox_to_anchor=(1.05, 1.0), loc='upper left')
    plt.show()

There are no clear effects of either of the relative frequency statistics. However, we will still feed them in as predictors to check if they have any additional effect. Thunderstorm and Tornado will be excluded, since they barely occur at all and Thunderstorm is already encoded as a day_thunder for the climatological days.

In [ ]:
mean_weather_climat_rel_features = ['lag_07', 'lag_365', 'month_sin', 'month_cos', 'day_of_week', 'school_holiday', 'public_holiday',
                                    'temp_mean', 'humidity_mean', 'rain_1h_mean', 'snow_1h_mean', 'day_frosty', 'day_thunder', 'day_clear','day_hazy', 'day_summer',
                                    'clear_total', 'cloudy_total', 'foggy_total', 'rainy_total', 'snowy_total']

In [ ]:
scores_lgbm_climat_rel = LGBM_cv(df_train, grouping_vars=['branch', 'product'], target='turnover', features=mean_weather_climat_rel_features)

In [ ]:
scores_lgbm_climat_rel

Further including relative frequencies of weather states slightly improved the forecasting performance. We will also test the forecasting performance of LGBM model when only including basic aggregate weather statistics and relative frequencies (w/o climatological days).

### only basic weather statistics and relative frequency measures

In [ ]:
mean_weather_rel_features = ['lag_07', 'lag_365', 'month_sin', 'month_cos', 'day_of_week', 'school_holiday', 'public_holiday',
                                    'temp_mean', 'humidity_mean', 'rain_1h_mean', 'snow_1h_mean',
                                    'clear_total', 'cloudy_total', 'foggy_total', 'rainy_total', 'snowy_total']

In [ ]:
scores_lgbm_freq = LGBM_cv(df_train, grouping_vars=['branch', 'product'], target='turnover', features=mean_weather_rel_features)

In [ ]:
scores_lgbm_freq

Relative frequencies of weather states worsen the prediction when fed into cross-validation.

We will therefore continue with the daily mean weather statistics and climatological days, since these features resulted in the best performance on cross-validation.

### GridSearch on LightGBM
After identifying the best combination of features, we will perform hyperparameter tuning using GridSearch to further optimize LGBM forecasting performance. Specifically, we will test different boosting types, numbers of estimators, and different learning rates.

In [ ]:
# create hyperparameter dictionary for grid search
lgbm_params = {
    'boosting_type': ['gbdt', 'dart'],
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.05, 0.08, 0.1]
}

In [ ]:
# initialize df for storing results from gridsearch
grid_results = pd.DataFrame({'boosting_type': [], 'n_estimators': [], 'learning_rate': [], 'MAPE': []})

for i, params in enumerate(product(lgbm_params['boosting_type'], lgbm_params['n_estimators'], lgbm_params['learning_rate'])):
    print(params)

    local_params = {
        'boosting_type': params[0],
        'n_estimators': int(params[1]),
        'learning_rate': float(params[2])}
    
                
    lgbm = LGBMRegressor(objective='regression', importance_type='gain', random_state=42, **local_params)
    
    # initialize empty list to compute average MAPE overall individual time series per hyperparameter configuration
    mapes_local = []

     # iterate over all individual series and perform cross-validation
    for k, group in enumerate(product(df_train['branch'].unique(), df_train['product'].unique())):
        # subselect time series and generate lag features
        ts = df_train[(df_train['branch']==group[0]) & (df_train['product']==group[1])].copy()

        # perform cross validation
        tss = TimeSeriesSplit(n_splits=52, test_size=7, gap=0)

        fold=0
        scores = []
        for train_i, val_i in tss.split(ts):

            train = ts.iloc[train_i]
            val = ts.iloc[val_i]
            features = ['lag_07', 'lag_365', 'month_sin', 'month_cos', 'day_of_week', 'school_holiday', 'public_holiday',
                            'temp_mean', 'humidity_mean', 'rain_1h_mean', 'snow_1h_mean',
                                                    'day_frosty', 'day_thunder', 'day_clear','day_hazy', 'day_summer']
                    
            X_train = train[features]
            X_val = val[features]
            y_train = train['turnover']
            y_val = val['turnover']

            lgbm.fit(X_train, y_train)
            y_pred = pd.Series(lgbm.predict(X_val))
            # correct for holiday effects in validation set if necessary
            # if holiday is in validation set, drop elements at corresponding index position in both y_val and y_pred
            if 1 in y_val.unique():
                idx_val = [i for i in range(len(y_val.tolist())) if y_val.tolist()[i]==1]
                y_val = y_val.drop(y_val.index[idx_val])
                y_pred = y_pred.drop(y_pred.index[idx_val])

            mape = mean_absolute_percentage_error(y_val, y_pred)
            scores.append(mape)
        
        # calculate mean MAPE score for individual time series
        mean_score = np.mean(scores)
        mapes_local.append(mean_score)
    
    grid_results.loc[i, 'boosting_type'] = params[0]
    grid_results.loc[i, 'n_estimators'] = params[1]
    grid_results.loc[i, 'learning_rate'] = params[2]
    grid_results.loc[i, 'MAPE'] = np.mean(mapes_local)

In [ ]:
grid_results

In [ ]:
grid_results.to_csv('../models/LGBM_hyperparams.csv', index=False)